In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait # 화면이 나올때까지 대기
from selenium.webdriver.support import expected_conditions as EC    # 화면상태체크
import requests
from bs4 import BeautifulSoup
import time
import os
import csv
import re

In [49]:
url = "https://www.bluer.co.kr/search?query=&foodTypeDetail=137&zone1=%EC%84%9C%EC%9A%B8%20%EA%B0%95%EB%82%A8&food2=%ED%8C%8C%EC%8A%A4%ED%83%80"    # 파스타

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Referer": "www.bluer.co.kr"
}
browser = webdriver.Chrome()    # 창열기
browser.maximize_window() # 최대창으로 확대
browser.get(url)

In [ ]:
# 필요없는 태그도 긁어옴(사용x)
soup = BeautifulSoup(browser.page_source, "lxml")
ul = soup.find("ul",{"id":"list-restaurant"})
# elem = browser.find_element(By.XPATH,'//*[@id="layer"]/button[2]')  # 광고창 제거
# elem.click()
lis = ul.find_all("li")
for i, li in enumerate(lis):
    target_div = li.select_one(".embed-responsive-item")
    image_url=""
    if target_div and 'style' in target_div.attrs:
        style_str = target_div['style']
        
        # 2. 정규표현식으로 url("...") 부분만 추출
        match = re.search(r'url\("?(.*?)"?\)', style_str)
        if match:
            image_url = match.group(1)
            
            # 3. 상대 경로인 경우 도메인 주소 결합 (필요 시)
            if image_url.startswith('/'):
                image_url = "https://www.bluer.co.kr" + image_url

            try:
                # 2. 이미지 데이터 요청 (headers 추가하여 403 방지)
                response = requests.get(image_url, headers=headers)

                if response.status_code == 200:
                    # 3. 파일 이름 추출 (URL의 마지막 부분 사용)
                    filename = image_url.split('/')[-1]
                    # 특수문자 제거 또는 간단하게 i 번호 사용 가능
                    save_path = os.path.join("images", f"{i}_{filename}")

                    with open(save_path, "wb") as f:
                        f.write(response.content)
                    print(f"   => {i}번째 저장 완료: {save_path}")
                else:
                    print(f"   => 다운로드 실패 (상태 코드: {response.status_code})")
            except Exception as e:
                print(f"   => 에러 발생: {e}")
        
    # 1. 식당 이름
    title = li.select_one("h3").get_text(strip=True) if li.select_one("h3") else ""
    
    # 2. 주소
    address = li.select_one(".juso-info").get_text(strip=True) if li.select_one(".juso-info") else ""
    
    # 3. 영업 상태
    status_div = li.select_one(".time-info .content-info")
    status = status_div.get_text(" ", strip=True) if status_div else ""
    
    # 4. 음식 종류
    food_types = [ft.get_text(strip=True) for ft in li.select(".foodtype li")]
    food_type_str = ", ".join(food_types)
    
    print(f"{i+1}번째 이미지 주소: {image_url}")
    print(f"\n[{i+1}번째 식당 정보]")
    print(f"상호명: {title}")
    print(f"주소: {address}")
    print(f"영업상태: {status}")
    print(f"종류: {food_type_str}")

In [ ]:
output_index = 1  # 출력용 인덱스 (연속 번호)
while True:
    soup = BeautifulSoup(browser.page_source, "lxml")
    ul = soup.find("ul", {"id": "list-restaurant"})
    lis = ul.find_all("li")

    for li in lis:
        # 식당 정보
        title = li.select_one("h3").get_text(strip=True) if li.select_one("h3") else ""
        address = li.select_one(".juso-info").get_text(strip=True) if li.select_one(".juso-info") else ""
        status_div = li.select_one(".time-info .content-info")
        status = status_div.get_text(" ", strip=True) if status_div else ""
        food_types = [ft.get_text(strip=True) for ft in li.select(".foodtype li")]
        food_type_str = ", ".join(food_types)

        # 정보가 없는 경우는 건너뜀
        if not any([title, address, status, food_type_str]):
            continue

        # 이미지
        target_div = li.select_one(".embed-responsive-item")
        image_url = ""
        if target_div and 'style' in target_div.attrs:
            style_str = target_div['style']
            match = re.search(r'url\("?(.*?)"?\)', style_str)
            if match:
                image_url = match.group(1)
                # 플레이스홀더 이미지 제외
                if image_url.startswith('/assets/image/restaurant/img_restaruant_no_image.png'):
                    image_url = ""
                elif image_url.startswith('/'):
                    image_url = "https://www.bluer.co.kr" + image_url

        # 이미지 다운로드 (있을 때만)
        if image_url:
            try:
                response = requests.get(image_url, headers=headers)
                if response.status_code == 200:
                    filename = image_url.split('/')[-1]
                    save_path = os.path.join("images", f"{output_index}_{filename}")
                    with open(save_path, "wb") as f:
                        f.write(response.content)
                    print(f"   => {output_index}번째 저장 완료: {save_path}")
                else:
                    print(f"   => 다운로드 실패 (상태 코드: {response.status_code})")
            except Exception as e:
                print(f"   => 에러 발생: {e}")

        print(f"{output_index}번째 이미지 주소: {image_url}")
        print(f"\n[{output_index}번째 식당 정보]")
        print(f"상호명: {title}")
        print(f"주소: {address}")
        print(f"영업상태: {status}")
        print(f"종류: {food_type_str}")

        output_index += 1
    # 3️⃣ 다음 페이지 버튼 찾기
    try:
        next_btn = browser.find_element(By.XPATH, '//*[@id="page-selection"]/ul/li[@class="next"]/a')
        # 클릭 가능할 때만 클릭
        if 'disabled' in next_btn.get_attribute('class'):
            print("마지막 페이지 도달")
            break
        next_btn.click()
        time.sleep(2)  # 페이지 로딩 대기 (필요하면 조정)
    except Exception as e:
        print("다음 페이지 없음 또는 클릭 실패:", e)
        break

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0"
}

# 이미지 저장 폴더
os.makedirs("images", exist_ok=True)

# CSV 헤더 정의
csv_file = "restaurants.csv"
header = ["resno", "location_id", "locationDetail_id", "res_nm", "addr", "desc",
          "phone", "open_time", "open_date", "sns_link", "image_url"]

with open(csv_file, mode="w", newline="", encoding="utf-8-sig") as f:
    writer = csv.writer(f)
    writer.writerow(header)  # 헤더 작성

    output_index = 1  # 출력용 인덱스 (연속 번호)

    while True:
        soup = BeautifulSoup(browser.page_source, "lxml")
        ul = soup.find("ul", {"id": "list-restaurant"})
        lis = ul.find_all("li")

        for li in lis:
            title = li.select_one("h3").get_text(strip=True) if li.select_one("h3") else ""
            address = li.select_one(".juso-info").get_text(strip=True) if li.select_one(".juso-info") else ""
            status_div = li.select_one(".time-info .content-info")
            status = status_div.get_text(" ", strip=True) if status_div else ""
            food_types = [ft.get_text(strip=True) for ft in li.select(".foodtype li")]
            food_type_str = ", ".join(food_types)

            # 정보가 없는 경우 건너뜀
            if not any([title, address, status, food_type_str]):
                continue

            # 상세페이지 URL
            thumb_div = li.select_one(".thumb-restaurant")
            detail_url = "https://www.bluer.co.kr" + thumb_div['data-href'] if thumb_div else ""

            # 상세페이지 데이터 초기화
            phone, address_detail, open_time, open_date, sns_link = "", "", "", "", ""
            if detail_url:
                try:
                    response = requests.get(detail_url, headers=headers)
                    if response.status_code == 200:
                        detail_soup = BeautifulSoup(response.text, "lxml")

                        # 전화번호
                        phone_tag = detail_soup.select_one('.icon_phone + .link')
                        phone = phone_tag.get_text(strip=True) if phone_tag else ""

                        # 상세주소
                        addr_tag = detail_soup.select_one('.icon_location + .content')
                        address_detail = addr_tag.get_text(strip=True) if addr_tag else ""

                        # 영업시간
                        time_tag = detail_soup.select_one('.time-info .content .extra')
                        open_time = time_tag.get_text(" ", strip=True) if time_tag else ""

                        # 개업일
                        open_date_tag = detail_soup.select_one('.icon_reservation_link + .content')
                        open_date = open_date_tag.get_text(strip=True).replace("개업일", "") if open_date_tag else ""

                        # SNS 링크
                        sns_tag = detail_soup.select_one('.icon_instagram + .link')
                        sns_link = sns_tag.get('href') if sns_tag else ""
                except Exception as e:
                    print("상세페이지 요청 실패:", e)

            # 이미지 가져오기
            target_div = li.select_one(".embed-responsive-item")
            image_url = ""
            if target_div and 'style' in target_div.attrs:
                style_str = target_div['style']
                match = re.search(r'url\("?(.*?)"?\)', style_str)
                if match:
                    image_url = match.group(1)
                    if image_url.startswith('/assets/image/restaurant/img_restaruant_no_image.png'):
                        image_url = ""
                    elif image_url.startswith('/'):
                        image_url = "https://www.bluer.co.kr" + image_url

            # 이미지 다운로드
            if image_url:
                try:
                    img_response = requests.get(image_url, headers=headers)
                    if img_response.status_code == 200:
                        filename = image_url.split('/')[-1]
                        save_path = os.path.join("images", f"{output_index}_{filename}")
                        with open(save_path, "wb") as f_img:
                            f_img.write(img_response.content)
                        print(f"   => {output_index}번째 이미지 저장 완료: {save_path}")
                    else:
                        print(f"   => 이미지 다운로드 실패: {image_url}")
                except Exception as e:
                    print("이미지 다운로드 에러:", e)

            # CSV 작성
            writer.writerow([
                output_index,
                1,  # location_id
                1,  # locationDetail_id
                title,
                address_detail or address,
                f"{status} / {food_type_str}",
                phone,
                open_time,
                open_date,
                sns_link,
                image_url
            ])

            print(f"{output_index}번째 식당 저장 완료: {title}")
            output_index += 1

        # 다음 페이지 클릭
        try:
            next_btn = browser.find_element(By.XPATH, '//*[@id="page-selection"]/ul/li[@class="next"]/a')
            if 'disabled' in next_btn.get_attribute('class'):
                print("마지막 페이지 도달")
                break
            next_btn.click()
            time.sleep(2)
        except Exception as e:
            print("다음 페이지 없음 또는 클릭 실패:", e)
            break

print("CSV 저장 완료:", csv_file)

In [ ]:
# === CSV 및 폴더 설정 ===
import os, csv, time, re, requests
from bs4 import BeautifulSoup

os.makedirs("output", exist_ok=True)
os.makedirs("images", exist_ok=True)

restaurants_csv = "output/restaurants.csv"
times_csv = "output/operating_times.csv"
images_csv = "output/images.csv"
menus_csv = "output/menus.csv"

headers = {"User-Agent": "Mozilla/5.0"}

# CSV 헤더 작성
for file, header in [(restaurants_csv, ["resno","location_id","locationDetail_id","res_nm","addr","desc","tel"]),
                     (times_csv, ["resno","week","open_time","close_time","desc"]),
                     (images_csv, ["resno","img_url","img_nm"]),
                     (menus_csv, ["resno","ftypeno","fnm","price"])]:
    with open(file, "w", newline="", encoding="utf-8-sig") as f:
        csv.writer(f).writerow(header)

# 시작 인덱스
resno_index = 1

while True:
    soup = BeautifulSoup(browser.page_source, "lxml")
    ul = soup.find("ul", {"id": "list-restaurant"})
    if not ul:
        print("식당 목록이 없습니다.")
        break
    lis = ul.find_all("li")
    if not lis:
        print("식당 li 태그가 없습니다.")
        break

    for li in lis:
        # 기본 정보
        title = li.select_one("h3").get_text(strip=True) if li.select_one("h3") else ""
        address = li.select_one(".juso-info").get_text(strip=True) if li.select_one(".juso-info") else ""
        food_types = [ft.get_text(strip=True) for ft in li.select(".foodtype li")]
        food_type_str = ", ".join(food_types)

        # 상세페이지 URL
        thumb_div = li.select_one(".thumb-restaurant")
        detail_url = None
        if thumb_div and thumb_div.has_attr("data-href"):
            detail_url = "https://www.bluer.co.kr" + thumb_div["data-href"]

        tel = ""

        if detail_url:
            try:
                browser.get(detail_url)
                time.sleep(1)
                detail_soup = BeautifulSoup(browser.page_source, "lxml")
            except Exception as e:
                print(f"상세페이지 접근 실패: {detail_url} / {e}")
                continue

            # 전화번호
            tel_tag = detail_soup.select_one("div.info-item img[src*='icon_phone'] + a")
            tel = tel_tag.get_text(strip=True) if tel_tag else ""

            # 운영시간
            time_div = detail_soup.select_one("div.info-item.time-info .extra.on")
            if time_div:
                times_text = time_div.get_text("<br>", strip=True).split("<br>")
                for t_line in times_text:
                    if "~" in t_line and any(c.isdigit() for c in t_line):
                        m = re.match(r"(.+?)\s+(\d{1,2}:\d{2})~(\d{1,2}:\d{2})", t_line)
                        if m:
                            week, open_time, close_time = m.groups()
                        else:
                            week, open_time, close_time = t_line, "", ""
                        desc_line = ""
                        with open(times_csv, "a", newline="", encoding="utf-8-sig") as f:
                            csv.writer(f).writerow([resno_index, week, open_time, close_time, desc_line])

            # 이미지
            # 상세페이지 이미지
            owl_stage = detail_soup.select_one("div.owl-stage")
            if owl_stage:
                imgs = owl_stage.select("img")
                for img_tag in imgs:
                    img_url = img_tag.get("src")
                    if img_url:
                        # 절대 경로 처리
                        if img_url.startswith("/"):
                            img_url = "https://www.bluer.co.kr" + img_url

                        # 파일 이름 및 저장 경로
                        img_nm = f"{resno_index}_{img_url.split('/')[-1]}"
                        save_path = os.path.join("images", img_nm)

                        # 이미지 다운로드
                        try:
                            resp = requests.get(img_url, headers=headers)
                            if resp.status_code == 200:
                                with open(save_path, "wb") as f_img:
                                    f_img.write(resp.content)
                                print(f"   => 상세페이지 이미지 저장 완료: {save_path}")
                            else:
                                print(f"   => 이미지 다운로드 실패: {img_url}")
                        except Exception as e:
                            print(f"   => 이미지 다운로드 에러: {img_url} / {e}")

                        # images.csv 기록 (DB에 경로로 저장)
                        with open(images_csv, "a", newline="", encoding="utf-8-sig") as f:
                            csv.writer(f).writerow([resno_index, save_path, img_nm])

            # 메뉴
            menu_list = detail_soup.select("ul.restaurant-menu-list li.item-menu")
            for menu_li in menu_list:
                fnm_tag = menu_li.select_one("div.content .title")
                price_tag = menu_li.select_one("div.content .price")
                if fnm_tag and price_tag:
                    fnm = fnm_tag.get_text(strip=True)
                    price_text = price_tag.get_text(strip=True).replace("원","").replace(",","")
                    price = int(price_text) if price_text.isdigit() else 0
                    with open(menus_csv, "a", newline="", encoding="utf-8-sig") as f:
                        csv.writer(f).writerow([resno_index, 1, fnm, price])  # ftypeno=1

            browser.back()
            time.sleep(1)

        # CSV 기본 정보 저장
        with open(restaurants_csv, "a", newline="", encoding="utf-8-sig") as f:
            csv.writer(f).writerow([
                resno_index,
                1,
                1,
                title,
                address,
                food_type_str,
                tel
            ])

        print(f"{resno_index} - {title} 저장 완료")
        resno_index += 1

    # 다음 페이지
    try:
        next_btn = browser.find_element(By.XPATH, '//*[@id="page-selection"]/ul/li[@class="next"]/a')
        if 'disabled' in next_btn.get_attribute('class'):
            print("마지막 페이지 도달")
            break
        next_btn.click()
        time.sleep(2)
    except Exception as e:
        print("다음 페이지 없음 또는 클릭 실패:", e)
        break

browser.quit()
print("모든 데이터 CSV 저장 완료!")


In [50]:
# === 라이브러리 및 폴더 설정 ===
import os, csv, time, re, requests
from bs4 import BeautifulSoup

# 폴더 생성
os.makedirs("output", exist_ok=True)
os.makedirs("images", exist_ok=True)

# CSV 파일 경로
restaurants_csv = "output/restaurants.csv"
times_csv = "output/operating_times.csv"
images_csv = "output/images.csv"
menus_csv = "output/menus.csv"

headers = {"User-Agent": "Mozilla/5.0"}

# CSV 헤더 작성
csv_headers = [
    (restaurants_csv, ["resno","location_id","locationDetail_id","res_nm","addr","desc","tel"]),
    (times_csv, ["resno","week","open_time","close_time","desc"]),
    (images_csv, ["resno","img_url","img_nm"]),
    (menus_csv, ["resno","ftypeno","fnm","price"])
]
for file, header in csv_headers:
    with open(file, "w", newline="", encoding="utf-8-sig") as f:
        csv.writer(f).writerow(header)

# 시작 resno
resno_index = 1

# === 크롤링 시작 ===
while True:
    soup = BeautifulSoup(browser.page_source, "lxml")
    ul = soup.find("ul", {"id": "list-restaurant"})
    if not ul:
        print("식당 목록이 없습니다.")
        break

    lis = ul.find_all("li")
    if not lis:
        print("식당 li 태그가 없습니다.")
        break

    for li in lis:
        # 기본 정보
        title = li.select_one("h3").get_text(strip=True) if li.select_one("h3") else ""
        address = li.select_one(".juso-info").get_text(strip=True) if li.select_one(".juso-info") else ""
        food_types = [ft.get_text(strip=True) for ft in li.select(".foodtype li")]
        food_type_str = ", ".join(food_types)

        if not title and not address:
            print("식당 정보 없음, 건너뜀")
            continue  # 진짜 없는 경우만 건너뜀

        # 상세페이지 URL
        thumb_div = li.select_one(".thumb-restaurant")
        detail_url = "https://www.bluer.co.kr" + thumb_div["data-href"] if thumb_div and thumb_div.has_attr("data-href") else None

        # 초기값
        tel, open_time, open_date = "", "", ""
        sns_link = ""
        images_list = []
        menus_list = []

        # 상세페이지 접속
        if detail_url:
            try:
                browser.get(detail_url)
                time.sleep(1)
                detail_soup = BeautifulSoup(browser.page_source, "lxml")

                # 전화번호
                tel_tag = detail_soup.select_one("div.info-item img[src*='icon_phone'] + a")
                tel = tel_tag.get_text(strip=True) if tel_tag else ""

                # 운영시간
                time_div = detail_soup.select_one("div.info-item.time-info .extra.on")
                if time_div:
                    times_text = time_div.get_text("<br>", strip=True).split("<br>")
                    for t_line in times_text:
                        if "~" in t_line and any(c.isdigit() for c in t_line):
                            m = re.match(r"(.+?)\s+(\d{1,2}:\d{2})~(\d{1,2}:\d{2})", t_line)
                            week, open_t, close_t = m.groups() if m else (t_line, "", "")
                            with open(times_csv, "a", newline="", encoding="utf-8-sig") as f:
                                csv.writer(f).writerow([resno_index, week, open_t, close_t, ""])

                # 이미지
                # 상세페이지 이미지 (슬라이드 background-image)
                owl_stage = detail_soup.select_one("div.owl-stage")
                if owl_stage:
                    img_lis = owl_stage.select("li.item.restaurant-img.bg-cover")
                    for img_li in img_lis:
                        style = img_li.get("style", "")
                        match = re.search(r'url\((.*?)\)', style)
                        if match:
                            img_url = match.group(1).strip(' "\'')
                            # 절대 경로 처리
                            if img_url.startswith("/"):
                                img_url = "https://www.bluer.co.kr" + img_url

                            # 파일 이름 및 저장 경로
                            img_nm = f"{resno_index}_{img_url.split('/')[-1]}"
                            save_path = os.path.join("images", img_nm)

                            # 이미지 다운로드
                            try:
                                resp = requests.get(img_url, headers=headers)
                                if resp.status_code == 200:
                                    with open(save_path, "wb") as f_img:
                                        f_img.write(resp.content)
                                    print(f"   => 상세페이지 이미지 저장 완료: {save_path}")
                                else:
                                    print(f"   => 이미지 다운로드 실패: {img_url}")
                            except Exception as e:
                                print(f"   => 이미지 다운로드 에러: {img_url} / {e}")

                            # images.csv 기록
                            with open(images_csv, "a", newline="", encoding="utf-8-sig") as f:
                                csv.writer(f).writerow([resno_index, save_path, img_nm])



                # 메뉴
                menu_items = detail_soup.select("ul.restaurant-menu-list li.item-menu")
                for menu_li in menu_items:
                    fnm_tag = menu_li.select_one("div.content .title")
                    # 상세페이지 리뷰
                    desc_tag = detail_soup.select_one("div.restaurant-review-info .content")
                    desc = desc_tag.get_text(strip=True) if desc_tag else food_type_str  # 없으면 기존 food_type_str 사용
                    price_tag = menu_li.select_one("div.content .price")
                    if fnm_tag and price_tag:
                        fnm = fnm_tag.get_text(strip=True)
                        price_text = price_tag.get_text(strip=True).replace("원","").replace(",","")
                        price = int(price_text) if price_text.isdigit() else 0
                        with open(menus_csv, "a", newline="", encoding="utf-8-sig") as f:
                            csv.writer(f).writerow([resno_index, 1, fnm, price])  # ftypeno=1 (파스타)

                browser.back()
                time.sleep(1)

            except Exception as e:
                print(f"상세페이지 접속 실패: {detail_url} / {e}")
                # 상세페이지 실패해도 기본 정보 기록

        # 기본 정보 CSV 저장
        with open(restaurants_csv, "a", newline="", encoding="utf-8-sig") as f:
            csv.writer(f).writerow([
                resno_index, 1, 1, title, address, desc, tel
            ])

        print(f"{resno_index} - {title} 저장 완료")
        resno_index += 1

    # 다음 페이지
    try:
        next_btn = browser.find_element(By.XPATH, '//*[@id="page-selection"]/ul/li[@class="next"]/a')
        if 'disabled' in next_btn.get_attribute('class'):
            print("마지막 페이지 도달")
            break
        next_btn.click()
        time.sleep(2)
    except Exception as e:
        print("다음 페이지 없음 또는 클릭 실패:", e)
        break

browser.quit()
print("모든 CSV 저장 완료!")


   => 상세페이지 이미지 저장 완료: images\1_es_31bf4d8b581642a5b8021409037b0f30.jpg
   => 상세페이지 이미지 저장 완료: images\1_es_fc2ef8c1593a4f6388291f6802d31c7b.jpg
   => 상세페이지 이미지 저장 완료: images\1_es_005718e17fe94cdc926cafa5a8f8664e.jpg
   => 상세페이지 이미지 저장 완료: images\1_es_7743b91ceaee4ffaa6c8be2304758c64.jpg
   => 상세페이지 이미지 저장 완료: images\1_es_026f4336903b4dab9a7bc034e7f9497c.jpg
   => 상세페이지 이미지 저장 완료: images\1_es_1e0bbaf18a6645b1b7c2f6147340f054.jpg
   => 상세페이지 이미지 저장 완료: images\1_es_e23bc8881e2f4b58be8ba21c638a0b91.jpg
   => 상세페이지 이미지 저장 완료: images\1_es_d71991b717264202a25476f1db714c0f.jpg
   => 상세페이지 이미지 저장 완료: images\1_es_a09375b0f57f4b9fa2efe155895dee0b.jpg
   => 상세페이지 이미지 저장 완료: images\1_es_4583199037bc483dab6a926c1cf34559.jpg
   => 상세페이지 이미지 저장 완료: images\1_es_b07a0aa44d2e463596b5ddd499825273.jpg
   => 상세페이지 이미지 저장 완료: images\1_es_2c161bc304714ee59cbb25a2b0e93cad.jpg
   => 상세페이지 이미지 저장 완료: images\1_es_f5afc911f15542459e091092d9aed8df.jpg
   => 상세페이지 이미지 저장 완료: images\1_es_bd6bc20aea29430d87a289d8ee0d0

KeyboardInterrupt: 

In [3]:
browser.close()